<a href="https://colab.research.google.com/github/stephenlovoi/twitter-sentiment-analysis/blob/main/sentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
# spark_version = 'spark-3.0.2'
# os.environ['SPARK_VERSION']=spark_version

# # Install Spark and Java
# !apt-get update
# !apt-get install openjdk-11-jdk-headless -qq 
# !wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
# !tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
# !pip install -q findspark


# Set Environment Variables
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
# import findspark
# findspark.init()

--2021-06-18 23:35:33--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-06-18 23:35:33--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.57K  --.-KB/s    in 0s      

2021-06-18 23:35:33 (44.3 MB/s) - written to stdout [1608/1608]

setup Cola

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
import pyspark as ps
import warnings
from pyspark.sql import SQLContext
from pyspark.ml import Pipeline
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline


In [3]:
# Start Spark session
spark = sparknlp.start()

In [4]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar



--2021-06-18 23:36:05--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.4’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.76MB/s    in 0.5s    

2021-06-18 23:36:06 (1.76 MB/s) - ‘postgresql-42.2.9.jar.4’ saved [914037/914037]



In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://trump-twitter-bucket.s3.us-east-2.amazonaws.com/trump_archive.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("trump_archive.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
df.show()

+---+--------------------+---------+---------+--------+-------------------+
|_c0|                text|isRetweet|favorites|retweets|               date|
+---+--------------------+---------+---------+--------+-------------------+
|  0|Republicans and D...|        f|       49|     255|2011-08-02 18:07:48|
|  1|The Unsolicited M...|        f|    80527|   23502|2020-09-12 20:10:58|
|  2|The threshold ide...|        f|   493076|  100609|2020-11-09 19:36:26|
|  3|Nevada is turning...|        f|   363489|   78378|2020-11-09 19:54:33|
|  4|Wisconsin is look...|        f|   347994|   61006|2020-11-09 20:02:19|
|  5|Pennsylvania prev...|        f|   479292|   81458|2020-11-09 20:17:47|
|  6|Just landed in Oh...|        f|   139153|   14000|2020-10-24 20:13:08|
|  7|Georgia will be a...|        f|   637719|   94570|2020-11-09 20:21:43|
|  8|How can you certi...|        f|   211508|   56945|2021-01-04 15:07:16|
|  9|Polls numbers are...|        f|   280261|   52623|2020-10-16 11:24:43|
| 10|Heading

In [6]:
df = df[~df.text.startswith('"')]

In [7]:
text_list = list(df.select('text').toPandas()['text'])


In [8]:
# from pyspark.sql.functions import col
# df = df.where(col("_c0").between(0, 20))
# df = df.dropna()

# tweet_col = df.select('text').collect()
# tweet_col

In [9]:

MODEL_NAME='sentimentdl_use_twitter'
# text_list = [
#              """Republicans and Democrats have both created our economic problems.""",
#              """The Unsolicited Mail In Ballot Scam is a major threat to our Democracy, &amp; the Democrats know it. Almost all recent elections using this system, even though much smaller &amp;  with far fewer Ballots to count, have ended up being a disaster. Large numbers of missing Ballots &amp; Fraud!"""
#              ]
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(name=MODEL_NAME, lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

sentiment_df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = pipelineModel.transform(sentiment_df)

sentiment = result.select(F.explode(F.arrays_zip('document.result', 'sentiment.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("sentiment"))
        


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]


In [10]:
sentiment.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|document                                                                                                                                                                                                                                                                                          |sentiment|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|Republicans and Democrats have both created our economic problems.                        

In [11]:
tweets = df.select(["text","favorites", "date"])

In [12]:
joined = tweets.join(sentiment,tweets.text ==  sentiment.document,"inner")
joined.show()

+--------------------+---------+-------------------+--------------------+---------+
|                text|favorites|               date|            document|sentiment|
+--------------------+---------+-------------------+--------------------+---------+
|Republicans and D...|       49|2011-08-02 18:07:48|Republicans and D...| negative|
|The Unsolicited M...|    80527|2020-09-12 20:10:58|The Unsolicited M...| negative|
|The threshold ide...|   493076|2020-11-09 19:36:26|The threshold ide...|  neutral|
|Nevada is turning...|   363489|2020-11-09 19:54:33|Nevada is turning...| negative|
|Wisconsin is look...|   347994|2020-11-09 20:02:19|Wisconsin is look...| positive|
|Pennsylvania prev...|   479292|2020-11-09 20:17:47|Pennsylvania prev...| negative|
|Just landed in Oh...|   139153|2020-10-24 20:13:08|Just landed in Oh...| positive|
|Georgia will be a...|   637719|2020-11-09 20:21:43|Georgia will be a...| positive|
|How can you certi...|   211508|2021-01-04 15:07:16|How can you certi...| ne

In [13]:
from pyspark.sql.types import IntegerType
joined = joined.withColumn("favorites", joined["favorites"].cast(IntegerType()))
joined.show()

+--------------------+---------+-------------------+--------------------+---------+
|                text|favorites|               date|            document|sentiment|
+--------------------+---------+-------------------+--------------------+---------+
|Republicans and D...|       49|2011-08-02 18:07:48|Republicans and D...| negative|
|The Unsolicited M...|    80527|2020-09-12 20:10:58|The Unsolicited M...| negative|
|The threshold ide...|   493076|2020-11-09 19:36:26|The threshold ide...|  neutral|
|Nevada is turning...|   363489|2020-11-09 19:54:33|Nevada is turning...| negative|
|Wisconsin is look...|   347994|2020-11-09 20:02:19|Wisconsin is look...| positive|
|Pennsylvania prev...|   479292|2020-11-09 20:17:47|Pennsylvania prev...| negative|
|Just landed in Oh...|   139153|2020-10-24 20:13:08|Just landed in Oh...| positive|
|Georgia will be a...|   637719|2020-11-09 20:21:43|Georgia will be a...| positive|
|How can you certi...|   211508|2021-01-04 15:07:16|How can you certi...| ne

In [14]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.ml import Pipeline

pos_neg_to_num = StringIndexer(inputCol='sentiment', outputCol='label')
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="token_text", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

# Calculate term frequency 

# feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'favorites'], outputCol='features').setHandleInvalid("skip")
pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [15]:
fit = pipeline.fit(joined)
transform = fit.transform(joined)

transform.show()

+--------------------+---------+-------------------+--------------------+---------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|favorites|               date|            document|sentiment|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|
+--------------------+---------+-------------------+--------------------+---------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|Republicans and D...|       49|2011-08-02 18:07:48|Republicans and D...| negative|  1.0|[republicans, and...|[republicans, dem...|(262144,[4629,879...|(262144,[4629,879...|(262145,[4629,879...|
|The Unsolicited M...|    80527|2020-09-12 20:10:58|The Unsolicited M...| negative|  1.0|[the, unsolicited...|[unsolicited, mai...|(262144,[4376,462...|(262144,[4376,462...|(262145,[4376,462...|
|The threshold ide...|   

In [16]:
transform.select('label','features').show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(262145,[4629,879...|
|  1.0|(262145,[4376,462...|
|  2.0|(262145,[27576,58...|
|  1.0|(262145,[24980,26...|
|  0.0|(262145,[12336,49...|
|  1.0|(262145,[39561,46...|
|  0.0|(262145,[8538,927...|
|  0.0|(262145,[30950,45...|
|  1.0|(262145,[8538,879...|
|  0.0|(262145,[10564,39...|
|  0.0|(262145,[8538,275...|
|  0.0|(262145,[30480,67...|
|  1.0|(262145,[19036,25...|
|  1.0|(262145,[9056,270...|
|  0.0|(262145,[2977,462...|
|  0.0|(262145,[89356,92...|
|  1.0|(262145,[11262,25...|
|  0.0|(262145,[6203,153...|
|  0.0|(262145,[27576,73...|
|  1.0|(262145,[12409,28...|
+-----+--------------------+
only showing top 20 rows



In [17]:
from pyspark.ml.classification import NaiveBayes

# split train test  
training, testing = transform.randomSplit([0.7, 0.3])

# fit 
nb = NaiveBayes()
predictor = nb.fit(training)

# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show()


+--------------------+---------+-------------------+--------------------+---------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                text|favorites|               date|            document|sentiment|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------------------+---------+-------------------+--------------------+---------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|#2.  Keep your mo...|      372|2013-12-23 18:41:29|#2.  Keep your mo...| positive|  0.0|[#2., , keep, you...|[#2., , keep, mom...|(262144,[5528,244...|(262144,[5528,244...|(262145,[5528,244...|[-980.49539330993...|[0.99990739272697...|       0.0|
|#2. Be 

In [18]:
# Import test quotes 

# Run test quotes on model 

# Get accuracy

In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting was: %f" % acc)

Accuracy of model at predicting was: 0.560752


In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
x = joined.toPandas()

In [ ]:
# Save File as CSV
x.to_csv('trump_sentiment.csv')
!cp trump_sentiment.csv "drive/My Drive/"